# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

In [5]:
! pip install -q "scikit-learn~=1.6" \
    "datasets==3.3.2" "huggingface-hub==0.29.1" \
    "transformers==4.49.0" "evaluate==0.4.3" \
    "peft==0.14.0"


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [36]:

dataset_name = "SetFit/ag_news"
dataset_splits = ["train", "test"]

dataset_id2label={ 0: "World", 1: "Sports", 2: "Business", 3: "Sci/Tech" }
dataset_label2id={v:k for k, v in dataset_id2label.items()}

untuned_model_name = "openai-community/gpt2"

lora_tuned_path="./gpt-lora-tuned"

print(f"{dataset_id2label=}")
print(f"{dataset_label2id=}")

dataset_id2label={0: 'World', 1: 'Sports', 2: 'Business', 3: 'Sci/Tech'}
dataset_label2id={'World': 0, 'Sports': 1, 'Business': 2, 'Sci/Tech': 3}


## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [37]:
from datasets import load_dataset

# the ag_news dataset is split into 120k train rows and 7.6k test rows.
dataset = load_dataset(dataset_name)

# reduce record cound to fit in memory (and run faster)
dataset["train"] = dataset["train"].shuffle(seed=7).select(range(800))
dataset["test"] = dataset["test"].shuffle(seed=11).select(range(200))

# view the dataset characteristics
print("train =", dataset["train"])
print("train[0] =", dataset["train"][0])
print("--------------------")
print("test =", dataset["test"])
print("test[0] =", dataset["test"][0])


train = Dataset({
    features: ['text', 'label', 'label_text'],
    num_rows: 800
})
train[0] = {'text': 'Air France-KLM Sales Rise 6.4 on Passenger Increase (Update1) Air France-KLM Group, Europe #39;s biggest airline, said second-quarter sales grew 6.4 percent as passengers generated more revenue.', 'label': 2, 'label_text': 'Business'}
--------------------
test = Dataset({
    features: ['text', 'label', 'label_text'],
    num_rows: 200
})
test[0] = {'text': "Alcoa Warns Earnings to Miss Forecasts (Reuters) Reuters - Alcoa Inc. , the world's largest\\aluminum producer, on Thursday warned that third-quarter\\results would fall far short of Wall Street expectations, hurt\\by plant shutdowns, restructuring costs and weakness in some\\markets.", 'label': 2, 'label_text': 'Business'}


In [38]:
from transformers import AutoTokenizer

# https://huggingface.co/docs/transformers/en/model_doc/gpt2#usage-tips
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.padding_side = "right"
tokenizer.pad_token=tokenizer.eos_token
# tokenizer.add_special_tokens({'pad_token': '<|pad|>'})

# 
tokenized_dataset = {}
for split in dataset_splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda row: tokenizer(row["text"], truncation=True), batched=True
    )

# inspect the special token ids
print(f"{tokenizer.eos_token_id=}")
print(f"{tokenizer.pad_token_id=}")

# inspect the columns in the tokenized dataset
print(f"{tokenized_dataset['train']=}")
print(f"{tokenized_dataset['test']=}")

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

tokenizer.eos_token_id=50256
tokenizer.pad_token_id=50256
tokenized_dataset['train']=Dataset({
    features: ['text', 'label', 'label_text', 'input_ids', 'attention_mask'],
    num_rows: 800
})
tokenized_dataset['test']=Dataset({
    features: ['text', 'label', 'label_text', 'input_ids', 'attention_mask'],
    num_rows: 200
})


In [39]:
from transformers import AutoModelForSequenceClassification

untuned_model = AutoModelForSequenceClassification.from_pretrained(
    untuned_model_name,
    num_labels=len(dataset_id2label),
    id2label=dataset_id2label,
    label2id=dataset_id2label,
    pad_token_id=tokenizer.pad_token_id,
)

# ensure all the parameters of the base model are frozen.
# see: https://huggingface.co/transformers/v4.2.2/training.html
for param in untuned_model.base_model.parameters():
    param.requires_grad = False

print(untuned_model)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=4, bias=False)
)


In [40]:
import numpy as np
import evaluate
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorWithPadding

accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

training_args=TrainingArguments(
    output_dir="./data/gpt2-untuned",
    # set the learning rate
    learning_rate=0.005,
    # set the per device train batch size and eval batch size
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    # evaluate and save the model after each epoch
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
)
print(f"{training_args.device=}")

trainer = Trainer(
    model=untuned_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    processing_class=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)


training_args.device=device(type='mps')


In [41]:
# we are interested in the accuracy of the untuned gpt2 model.
trainer.evaluate()

{'eval_loss': 5.677774429321289,
 'eval_model_preparation_time': 0.0028,
 'eval_accuracy': 0.265,
 'eval_runtime': 3.012,
 'eval_samples_per_second': 66.401,
 'eval_steps_per_second': 16.6}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [42]:
from transformers import AutoTokenizer

# https://huggingface.co/docs/transformers/en/model_doc/gpt2#usage-tips
tokenizer = AutoTokenizer.from_pretrained(
    untuned_model_name,
    padding_side="right"
)
tokenizer.pad_token=tokenizer.eos_token
# tokenizer.add_special_tokens({'pad_token': '<|pad|>'})

def row_processor(row):
    inputs = tokenizer(
        row["text"],
        truncation=True, 
        padding="max_length", 
        max_length=256,
    )
    inputs['labels'] = inputs['input_ids'].copy()
    return inputs

# 
tokenized_dataset = {}
for split in dataset_splits:
    tokenized_dataset[split] = dataset[split].map(row_processor, batched=True)

# inspect the special token ids
print(f"{tokenizer.eos_token_id=}")
print(f"{tokenizer.pad_token_id=}")

# inspect the columns in the tokenized dataset
print(f"{tokenized_dataset['train']=}")
print(f"{tokenized_dataset['test']=}")

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

tokenizer.eos_token_id=50256
tokenizer.pad_token_id=50256
tokenized_dataset['train']=Dataset({
    features: ['text', 'label', 'label_text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 800
})
tokenized_dataset['test']=Dataset({
    features: ['text', 'label', 'label_text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 200
})


In [43]:
from peft import LoraConfig, TaskType, get_peft_model
from transformers import AutoModelForSequenceClassification

untuned_model = AutoModelForSequenceClassification.from_pretrained(
    untuned_model_name,
    num_labels=len(dataset_id2label),
    id2label=dataset_id2label,
    label2id=dataset_id2label,    
    pad_token_id=tokenizer.pad_token_id,
)

# ensure all the parameters of the base model are frozen.
# see: https://huggingface.co/transformers/v4.2.2/training.html
for param in gpt2_model.base_model.parameters():
    param.requires_grad = False

# for name, module in gpt2_model.named_modules():
#     print(name)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [44]:
lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS,
    fan_in_fan_out=True, # this is required for gpt2
    modules_to_save=["h.10"]
)

lora_model = get_peft_model(untuned_model, lora_config)

lora_model.print_trainable_parameters()
# lora_model

trainable params: 8,172,288 || all params: 132,615,168 || trainable%: 6.1624


In [45]:
import numpy as np
import evaluate
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorWithPadding

accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

training_args=TrainingArguments(
    output_dir="./data/news_gpt2_lora",
    # set the learning rate
    learning_rate=0.0025,
    # Set the per device train batch size and eval batch size
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    # evaluate and save the model after each epoch
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    weight_decay=0.02,
    load_best_model_at_end=True,
)
print(f"{training_args.device=}")

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    processing_class=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)


No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


training_args.device=device(type='mps')


In [46]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.338342,0.875000
2,No log,0.407811,0.875000
3,No log,0.382158,0.895000


TrainOutput(global_step=150, training_loss=0.4057020060221354, metrics={'train_runtime': 145.1669, 'train_samples_per_second': 16.533, 'train_steps_per_second': 1.033, 'total_flos': 343688085504000.0, 'train_loss': 0.4057020060221354, 'epoch': 3.0})

In [47]:

lora_model.save_pretrained(lora_tuned_path)


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [58]:
from peft import PeftConfig, PeftModel
#from peft import PeftModelForSequenceClassification
from transformers import AutoModelForSequenceClassification

peft_config = PeftConfig.from_pretrained(lora_tuned_path)
print(f"{peft_config.base_model_name_or_path=}")

peft_model = AutoModelForSequenceClassification.from_pretrained(
    peft_config.base_model_name_or_path,
    num_labels=len(dataset_id2label),
    id2label=dataset_id2label,
    label2id=dataset_id2label,
    pad_token_id=tokenizer.pad_token_id,
)

model_to_merge = PeftModel.from_pretrained(
    peft_model, 
    lora_tuned_path,
)

merged_model = model_to_merge.merge_and_unload()
merged_model.save_pretrained(lora_tuned_path + "-merged-model", merged_model)


peft_config.base_model_name_or_path='openai-community/gpt2'


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [59]:
gpt2_lora_model = AutoModelForSequenceClassification.from_pretrained(
    lora_tuned_path + "-merged-model",
    num_labels=len(dataset_id2label),
    id2label=dataset_id2label,
    label2id=dataset_id2label,    
    pad_token_id=tokenizer.pad_token_id,
)

# ensure all the parameters of the base model are frozen.
# see: https://huggingface.co/transformers/v4.2.2/training.html
for param in gpt2_merged_model.base_model.parameters():
    param.requires_grad = False

print(model)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=4, bias=False)
)


In [62]:

trainer = Trainer(
    model=gpt2_lora_model,
    eval_dataset=tokenized_dataset["test"],
    processing_class=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

In [63]:
trainer.evaluate()

{'eval_loss': 0.33834221959114075,
 'eval_model_preparation_time': 0.0033,
 'eval_accuracy': 0.875,
 'eval_runtime': 4.6394,
 'eval_samples_per_second': 43.109,
 'eval_steps_per_second': 5.389}